### Knapsack Problems

We start with the definition of the Knapsack problem, the variables involved and the algorithmic layout then we build on it as we go down.

We define an $Item$ for which properties (weight, value) of each item are constant.

In [1]:
import random
import math
from itertools import chain

In [2]:
class Item:
    def __init__(self, weight, value, quantity=0):
        self.weight = weight        # rho, pj
        self.value = value          # wj
        self.quantity = quantity    # how many of this item should be placed in Knapsack?
    
    def __sub__(self, other):
        return Item(self.weight, self.value - other.value)

### Bounded Knapsack Problem
Given a bound $h_j$ on identical copies of items $j$ available with weight $w_j$ amd $\rho_j$

maximize
\begin{equation}
f = \sum_{j=1}^{n} \rho_j k_j
\end{equation}
subject to
\begin{equation}
\sum_{j=1}^{n} w_j k_j \le C,
\end{equation}
where
\begin{equation}
0 \le k_j \le h_j, \\ 
h_j \in \mathbb{Z}, \\
j = 1, 2, 3 ... n
\end{equation}

This cannot be solved by any algorithm in polynomial time

### Binary Particle Swarm Optimization

# Paper Approach

## BKP GROA

In [3]:
from typing import Tuple
def bkp_groa(X: list, H: list, C: int) -> Tuple[list, float]:
    """
    X: Potential solution. List of Items
    H: Index of all items sorted in descending order.
    C: Knapsack capacity
    """    
    N = len(X)
    fweight = sum([x.weight*x.value for x in X])
    j = N - 1

    while fweight > C:
        x = X[H[j]]
        if x.value == 1:
            x.value = 0
            fweight -= x.weight
        j -= 1

    for j in range(N):
        x = X[H[j]]
        if x.value == 0 and fweight + x.weight <= C:
            x.value = 1
            fweight += x.weight
    
    return X, fweight

## Particle

In [4]:
class Particle(Item):
    def __init__(self, weight, value, quantity, dimension, vmax, c1, c2, W=[]):
        """
        Weight of Particle
        Value of Particle
        Dimension or Size of Population
        Vmax: used for initializing particle velocity
        $c_1, c_2$: Acceleration constants
        W:  weights
        """
        # $X = x_1, x_2, ... x_n$
        self.X = [self.create_item(W[i]) for i in range(dimension)] 
        # $V = v_1, v_2, ... v_n$
        self.velocity = [random.uniform(-vmax, vmax) for i in range(dimension)]
        # $P = p_1, p_2, ... p_n$ Algorithm 2, line 2
        self.local_best_position = self.X.copy()

        Item.__init__(self, weight, value)
        
        self.n = dimension
        self.c1 = c1
        self.c2 = c2

        self.fW = None
    
    def create_item(self, weight):
        """
        For initializing particles current position.
        """
        value = random.randint(0, 1)
        return Item(weight, value)

    @property
    def f(self):
        """
        returns the minimum position $p_i$
        """
        return min(self.local_best_position, key=lambda i: i.value)

    
    def update_current_velocity(self, global_best_position: float):
        """
        global_best_position: a scalar
        """
        self.pg = global_best_position
        velocity = [self.evaluate(r) for r in range(self.n)]
        self.velocity = velocity
    
    def evaluate(self, j):
        """
        Main Paper uses 
        $r_{1j}$ and $r_{2j}$ for q1, q2
        """
        q1 = random.uniform(0, 1)
        q2 = random.uniform(0, 1)

        return self.velocity[j] + \
            self.c1 * q1 * (self.local_best_position[j] - self.X[j]).value + \
            self.c2 * q2 * (self.pg - self.X[j]).value 
    
    def update_current_position(self):
        """"""
        self.q3 = random.uniform(0, 1)
        for j, item in enumerate(self.X):
            item.value = self.eval2(j)
    
    def eval2(self, j):
        sig = self.sigmoid(j)
        if sig <= self.q3:
            return 0
        else:
            return 1
    
    def sigmoid(self, j):
        k = self.velocity[j]
        return 1/(1 + math.exp(-k))
    
    @property
    def density(self):
        """
        performs $p_i/w_i$
        Where $p_i$ is gotten from self.f

        version 2 self.value/self.weight
        """
        return self.value/self.weight
    
    def update_local_best_position(self):
        """
        Equation 3 and algorithm seem different, using algorithm for now
        """
        f = self.f
        if self.f_X.value > f.value:
            self.local_best_position = self.X
        # else:
        #     doesn't change
        #     self.local_best_position = self.local_best_position
    
    @property
    def f_X(self):
        """
        $f(X_i)$
        Implemented as minimum of $X_i$
        """
        return min(self.X, key=lambda i: i.value)

## BPSO

#### Test Data

In [5]:
MaxIter = 2         # number of iterations
c1, c2 = 1.8, 1.8   # acceleration constants
vmax = 1.0          # Used in setting velocity

C = 141152 # Knapsack capacity
profits = [816,959,885,653,810,978,350,61,228,639,520,286,581,597,891,94,925,13,
          766,808,23,1003,482,65,865,196,357,840,623,344,424,457,924,354,433,
          347,627,302,672,179,574,419,172,592,420,920,566,784,432,660,57,91,88,
          624,791,727,227,824,312,459,918,475,759,1001,913,366,61,789,85,679,
          994,891,289,313,112,528,254,740,83,431,1007,151,454,135,309,688,576,
          955,218,114,391,530,389,817,963,440,397,338,230,173]

weights = [806,949,875,643,800,968,340,51,218,629,510,276,571,587,881,84,915,3,
          756,798,13,993,472,55,855,186,347,830,613,334,414,447,914,344,423,
          337,617,292,662,169,564,409,162,582,410,910,556,774,422,650,47,81,78,
          614,781,717,217,814,302,449,908,465,749,991,903,356,51,779,75,669,984,
          881,279,303,102,518,244,730,73,421,997,141,444,125,299,678,566,945,
          208,104,381,520,379,807,953,430,387,328,220,163]
bounds = [8,1,2,9,7,5,6,2,6,9,3,3,9,2,4,8,1,1,1,9,6,8,9,1,8,5,9,8,1,5,1,1,2,3,9,
         5,7,8,7,3,8,1,3,6,7,7,5,5,9,5,8,6,5,5,6,8,1,2,1,6,6,7,4,9,2,5,6,8,1,9,
         7,1,1,5,8,9,2,9,9,8,5,3,3,5,3,1,6,5,1,5,8,2,2,2,1,4,5,8,2,6]

#### Code

In [6]:
_profits = list(chain.from_iterable([[j]*bounds[i] for i, j in enumerate(profits)]))
_weights = list(chain.from_iterable([[j]*bounds[i] for i, j in enumerate(weights)]))
_bounds = [1]*sum(bounds)

# Population size, used in paper
N = len(_profits)
################################################################################
# N random particles
#(self, weight, value, quantity, dimension, vmax, z1, z2):
Ps = [Particle(w, p, b, N, vmax, c1, c2, _weights) for p, w, b in zip(_profits, _weights, _bounds)]

# TODO Instead of using H, we could sort Ps and access in accending or descending order.
H = [(i, j) for i, j in enumerate(Ps)]
H = sorted(H, key = lambda i: i[1].density, reverse=True) # where i is (index, particle)
H = [i for i, j in H]

# TODO calculate $P_g$
# global best position made up of the local best positions. It is thus N x N?
Pg = max(Ps, key=lambda p: p.f.value).local_best_position # List of Items
# print(Pg)
_d = sum([i.value for i in Pg])
print(f"Initial number of selected items based on best global position: {_d}")
# Optimal Solutions

t = 0

print("\nStarting optimisation.")
while (t <= MaxIter):
    print(f"Iteration: {t}")
    for i in range(N):
        particle = Ps[i]
        # update velocity
        particle.update_current_velocity(Pg[i])
        # update position
        particle.update_current_position()
        particle.X, particle.fW = bkp_groa(particle.X, H, C)
        
        particle.update_local_best_position()
    p = max([(i, p.f) for i, p in enumerate(Ps)], key=lambda j: j[1].value)
    p = Ps[p[0]]
    Pg = p.local_best_position
    t += 1 
print("Finished Optimisation.\n")
vals = [i.value for i in Pg]
print(f"{sum(vals)} items chosen out of {N} items")

_Ps = [p for p, _p in zip(Ps, vals) if _p == 1]
profs = [p.value for p in _Ps]
ws = [p.weight for p in _Ps]
print(f"Total profit of available items: {sum(_profits)}")
print(f"Sum of profits of items in knapsack: {sum(profs)}")
# From section 2 Paragraph 1 sum of weights should not exceed knapsack capacity
print(f"Sum of weights of items in knapsack: {sum(ws)}")
print(f"Capacity of Knapsack: {C}")

Initial number of selected items based on best global position: 237

Starting optimisation.
Iteration: 0
Iteration: 1
Iteration: 2
Finished Optimisation.

346 items chosen out of 499 items
Total profit of available items: 261631
Sum of profits of items in knapsack: 144051
Sum of weights of items in knapsack: 140591
Capacity of Knapsack: 141152
